- Main KITTI readme (old): https://github.com/yanii/kitti-pcl/blob/master/KITTI_README.TXT  
- About Intrinsics: http://ksimek.github.io/2013/08/13/intrinsic/
- Car scheme: http://www.cvlibs.net/publications/Geiger2013IJRR.pdf
- https://avisingh599.github.io/vision/visual-odometry-full/
- Image with reprojection: https://yadi.sk/i/JAIIsbP5dHAELg
- https://github.com/cgarg92/Stereo-visual-odometry

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData,  VisualOdometry, draw_matches, draw_keypoints
from shared.tools import find_max_clique

%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR)

In [ ]:
frame_idx = 150
c_l_img, c_r_img = dataset.get_color_images(frame_idx)
n_l_img, n_r_img = dataset.get_color_images(frame_idx+1)
Q_left = dataset.get_color_left_Q_matrix()
gt_transform = dataset._get_transform_mtrx(frame_idx)

print(gt_transform)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_l_img)
ax2.imshow(n_l_img)

In [ ]:
vo = VisualOdometry()
c_depth_frame = vo.process_depth(c_l_img, c_r_img, Q_left)
n_depth_frame = vo.process_depth(n_l_img, n_r_img, Q_left)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_depth_frame[:,:,2])
ax2.imshow(n_depth_frame[:,:,2])

In [ ]:
c_feats, n_feats = vo.get_features(c_l_img, n_l_img)

c_img_canvas = c_l_img.copy()
n_img_canvas = n_l_img.copy()

### Get 3D points 
c_pnts_3d, c_ft_idxs = vo.reproject_2d_to_3d_points(c_feats, c_depth_frame)
n_pnts_3d, n_ft_idxs = vo.reproject_2d_to_3d_points(n_feats, n_depth_frame)

ft_idxs = c_ft_idxs & n_ft_idxs

c_pnts_3d = c_pnts_3d[ft_idxs]
n_pnts_3d = n_pnts_3d[ft_idxs]
c_feats = c_feats[ft_idxs]
n_feats = n_feats[ft_idxs]

### Filter 
cl_idxs, _ = vo.max_clique_filter(c_pnts_3d, n_pnts_3d)
c_pnts_3d = c_pnts_3d[cl_idxs]
n_pnts_3d = n_pnts_3d[cl_idxs]
c_feats = c_feats[cl_idxs]
n_feats = n_feats[cl_idxs]

draw_keypoints(c_img_canvas, n_img_canvas, c_feats, n_feats)

# Rendering valid features
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

In [ ]:
# P_left, _ = dataset.get_color_P_matrix()
# print(P_left)

C_left, _ = dataset.get_color_С_matrix()
P_left = np.hstack((C_left, np.transpose([[0, 0, 0]])))

c_pnts_2d = vo.reproject_3d_to_2d(c_pnts_3d, P_left)
n_pnts_2d = vo.reproject_3d_to_2d(n_pnts_3d, P_left)

c_img_canvas = c_l_img.copy()
n_img_canvas = n_l_img.copy()

# Check after clique and projection
draw_keypoints(c_img_canvas, n_img_canvas, c_pnts_2d, n_pnts_2d)

# # Rendering valid features
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

## Define optimization functions

In [ ]:
from scipy.spatial.transform import Rotation as R
from scipy.optimize import least_squares

def get_transform_matrix(x):
    transform = np.eye(4)
    transform[:3,:3] = R.from_rotvec(x[:3]).as_matrix()
    transform[:3, 3] = x[3:]
    return transform
    
def from_transform_matrix(x):
    rotvec = R.from_matrix(x[:3,:3]).as_rotvec()
    transl = x[:3,3]
    return list([*rotvec, *transl])
    
def estimate_transform_2d(x, c_pnts_2d, n_pnts_2d, c_pnts_3d, n_pnts_3d, P_mtrx):
    """
        x - [rotvec, transform]
    """
    transform = get_transform_matrix(x)

    Proj_frwrd = P_mtrx @ np.linalg.inv(transform)
    Proj_bcwrd = P_mtrx @ transform
    
    n_pred_pnts_2d = vo.reproject_3d_to_2d(c_pnts_3d, Proj_frwrd)
    c_pred_pnts_2d = vo.reproject_3d_to_2d(n_pnts_3d, Proj_bcwrd)
    
    c_err = c_pnts_2d - c_pred_pnts_2d
    n_err = n_pnts_2d - n_pred_pnts_2d

    residual = np.vstack((c_err*c_err,n_err*n_err))
    return residual.flatten()

def estimate_transform_3d(x, c_pnts_3d, n_pnts_3d):
    """
        x - [rotvec, transform]
    """
    transform = get_transform_matrix(x)
    
    c_pred_pnts_3d = vo.transform_3d(transform, n_pnts_3d)
    n_pred_pnts_3d = vo.transform_3d(np.linalg.inv(transform), c_pnts_3d)
    
    c_err = np.linalg.norm(c_pnts_3d - c_pred_pnts_3d, axis=1)
    n_err = np.linalg.norm(n_pnts_3d - n_pred_pnts_3d, axis=1)

    residual = np.vstack((c_err,n_err))
    return residual.flatten()

In [ ]:
c_pnts_3d

In [ ]:
n_pnts_3d

In [ ]:
initial = from_transform_matrix(gt_transform)
print(initial)

errors = estimate_transform_2d(
    initial,
    c_pnts_2d,
    n_pnts_2d,
    c_pnts_3d,
    n_pnts_3d,
    P_left
)
print(errors)
print(np.sum(errors))

In [ ]:
initial = from_transform_matrix(gt_transform)
print(initial)

errors = estimate_transform_3d(
    initial,
    c_pnts_3d,
    n_pnts_3d
)

print(errors)
print(np.sum(errors))

## Optimization

In [ ]:
initial = np.zeros(6)
args = (
    c_pnts_2d,
    n_pnts_2d,
    c_pnts_3d,
    n_pnts_3d,
    P_left
)
optRes = least_squares(estimate_transform_2d, initial, method='lm', max_nfev=200, args=args, gtol=None, verbose=2)
optRes.x

In [ ]:
initial = np.zeros(6)
args = (
    c_pnts_3d,
    n_pnts_3d
)
optRes = least_squares(estimate_transform_3d, initial, method='lm', max_nfev=10000, args=args, verbose=2)
optRes.x

In [ ]:
С_left, _ = dataset.get_color_С_matrix()
n_pnts_2d = n_pnts_2d.astype(np.float32)
c_pnts_2d = c_pnts_2d.astype(np.float32)

c_pnts_3d = c_pnts_3d.astype(np.float32)
n_pnts_3d = n_pnts_3d.astype(np.float32)

retval, rvec, tvec = cv2.solvePnP(c_pnts_3d, n_pnts_2d, cameraMatrix=С_left, distCoeffs=None)

In [ ]:
c_pnts_3d

In [ ]:
retval

In [ ]:
rvec

In [ ]:
r_mtrx, _ = cv2.Rodrigues(rvec)
r_mtrx

In [ ]:
tvec

In [ ]:
c_1_pnts_2d, _ = cv2.projectPoints(c_pnts_3d, rvec, tvec, C_left, distCoeffs=None)
c_2_pnts_2d, _ = cv2.projectPoints(c_pnts_3d, np.zeros(3), np.zeros(3), C_left, distCoeffs=None)

print(c_pnts_2d)
print(c_1_pnts_2d)
print(c_2_pnts_2d)

In [ ]:
x = np.array([rvec, tvec], dtype=np.float32).reshape(6)
gt_x = from_transform_matrix(gt_transform)
gt_x = np.array(gt_x, dtype=np.float32)

print(x)
print(gt_x)

errors = estimate_transform_3d(
    x,
    c_pnts_new,
    n_pnts_new
)
errors